In [58]:
# limit for combination higgs
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections
import os
from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward as ak
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
# from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

print(sys.version)

3.6.8 (default, Nov  2 2021, 13:01:57) 
[GCC 8.4.1 20200928 (Red Hat 8.4.1-1)]


In [117]:
# base_path = "/storage/af/user/christiw/login-1/christiw/LLP/Run3/CMSSW_14_0_1/src/mds_analysis/"
# mc = uproot.open(base_path + 'data/raw/mc_2023.root')['MuonSystem']
# NEVENTS = uproot.open(base_path + 'data/raw/mc_2023.root')['NEvents'].values()

files = {}

base_path = "/storage/af/user/christiw/login-1/christiw/LLP/Run3/CMSSW_14_0_1/src/mds_analysis_240521/"
files['mc'] = uproot.open(base_path + 'data/processed/mc_pedro_hlt569_2023.root')['MuonSystem']
files['data'] = uproot.open(base_path + 'data/processed/r3_pedro_hlt569_2023.root')['MuonSystem']

# NEVENTS = uproot.open(base_path + 'data/raw/mc_2023.root')['NEvents'].values()


In [85]:
def L1_trg(cscClusterR, cscClusterZ, cscClusterSize):  
    first_in_ME11 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>100, cscClusterR<275), np.abs(cscClusterZ)>580), np.abs(cscClusterZ)<632) 
    first_in_ME12 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>275, cscClusterR<465), np.abs(cscClusterZ)>668), np.abs(cscClusterZ)<724)
    first_in_ME13 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>505, cscClusterR<700), np.abs(cscClusterZ)>668), np.abs(cscClusterZ)<724)
    first_in_ME21 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>139, cscClusterR<345), np.abs(cscClusterZ)>789), np.abs(cscClusterZ)<850)
    first_in_ME22 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>357, cscClusterR<700), np.abs(cscClusterZ)>791), np.abs(cscClusterZ)<850)
    first_in_ME31 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>160, cscClusterR<345), np.abs(cscClusterZ)>915), np.abs(cscClusterZ)<970)
    first_in_ME32 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>357, cscClusterR<700), np.abs(cscClusterZ)>911), np.abs(cscClusterZ)<970)
    first_in_ME41 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>178, cscClusterR<345), np.abs(cscClusterZ)>1002), np.abs(cscClusterZ)<1063)
    first_in_ME42 = np.logical_and(np.logical_and(np.logical_and(cscClusterR>357, cscClusterR<700), np.abs(cscClusterZ)>1002), np.abs(cscClusterZ)<1063)
    
    first_in_plateau_ME11 = np.logical_and(first_in_ME11, cscClusterSize>=500)
    first_in_plateau_ME21 = np.logical_and(first_in_ME21, cscClusterSize>=500)
    first_in_plateau_ME31 = np.logical_and(first_in_ME31, cscClusterSize>=500)
    first_in_plateau_ME41 = np.logical_and(first_in_ME41, cscClusterSize>=500)

    first_in_plateau_ME12 = np.logical_and(first_in_ME12, cscClusterSize>=200)
    first_in_plateau_ME13 = np.logical_and(first_in_ME13, cscClusterSize>=200)
    first_in_plateau_ME22 = np.logical_and(first_in_ME22, cscClusterSize>=200)
    first_in_plateau_ME32 = np.logical_and(first_in_ME32, cscClusterSize>=200)
    first_in_plateau_ME42 = np.logical_and(first_in_ME42, cscClusterSize>=200)
    
    first_in_plateau = first_in_plateau_ME11 | first_in_plateau_ME12 | first_in_plateau_ME13 | first_in_plateau_ME21 | first_in_plateau_ME22 | \
    first_in_plateau_ME31 | first_in_plateau_ME32 | first_in_plateau_ME41 | first_in_plateau_ME42
    return first_in_plateau

def deltaPhi( phi1,  phi2):
    dphi = phi1-phi2
    while np.count_nonzero(dphi > math.pi)>0:
        dphi[dphi > math.pi] -= 2*math.pi
    while np.count_nonzero(dphi< -math.pi)>0:
        dphi[dphi < -math.pi] += 2*math.pi
    return dphi

## CSC-CSC

In [112]:
%%time

for k,T in files.items():

    # weight = T['weight'].array()* 52*27000/NEVENTS
    if 'data' in k: weight = T['weight'].array()
    else: weight = T['weight'].array()* 1.1328524540090597e-06 * 27.82 * 1000 # from Pedro 2023
    #48/2.31671e+07= 2.0719037e-06?

    presel =  (T["nCscRechitClusters"].array() > 1)
    # cond = T["HLTDecision"].array()[:,566] & (T["nCscRechitClusters"].array() > 1)
    ########### SELECTION: JETS ############

    print("sample", '\t',np.sum(weight[presel]))

    # csc = (T['gLLP_eta'].array()<3) & (T['gLLP_decay_vertex_r'].array()<800)\
    # & (T['gLLP_decay_vertex_z'].array()<1200) & (T['gLLP_decay_vertex_z'].array()>400)
    sel_csc = T['cscRechitCluster_match_gLLP'].array() & (np.abs(T['cscRechitCluster_match_gLLP_eta'].array())<3) & (T['cscRechitCluster_match_gLLP_decay_r'].array()<800)\
    & (np.abs(T['cscRechitCluster_match_gLLP_decay_z'].array())<1200) & (np.abs(T['cscRechitCluster_match_gLLP_decay_z'].array())>400)

    if not 'data' in k:
        presel = presel & (np.sum(sel_csc, axis=1)>1)
        print("acceptance", '\t',np.sum(weight[presel]))

    cscRechitClusterR = (T['cscRechitClusterX'].array()**2+T['cscRechitClusterY'].array()**2)**0.5
    L1 = L1_trg(cscRechitClusterR, np.abs(T['cscRechitClusterZ'].array()), np.abs(T['cscRechitClusterSize'].array()))
    intime = (T['cscRechitClusterTimeWeighted'].array() < 12.5) & (T['cscRechitClusterTimeWeighted'].array()> -5) \
    & (T['cscRechitClusterTimeSpreadWeightedAll'].array() < 20)

    presel =presel & (np.sum(L1 & intime, axis=1)>0)
    print("L1 (at least one cluster)", '\t',np.sum(weight[presel]))

    presel = presel & (T['nDtRechitClusters'].array()>=0)
    print("0 DT", '\t',np.sum(weight[presel]))

    presel = presel & (T['nCscRechitClusters'].array()==2)
    print("1 CSC-CSC", '\t',np.sum(weight[presel]))

    intime = intime[presel]
    weight = weight[presel]
    cond = intime[:,0]
    print("CSC0IT", '\t',np.sum(weight[cond]))

    cond = cond & intime[:,1]
    print("CSC1IT", '\t',np.sum(weight[cond]))

    me1 = (T['cscRechitClusterNRechitChamberPlus11'].array()+ T['cscRechitClusterNRechitChamberPlus12'].array()+\
    T['cscRechitClusterNRechitChamberMinus11'].array()+ T['cscRechitClusterNRechitChamberMinus12'].array()==0)[presel]
    cond = cond & me1[:,0]
    print("CSC0 ME1", '\t',np.sum(weight[cond]))

    cond = cond & me1[:,1]
    print("CSC1 ME1", '\t',np.sum(weight[cond]))

    cond = cond & (T['met'].array()[presel]<150)
    print("low MET", '\t', np.sum(weight[cond]))

    cscRechitClusterPhi = T['cscRechitClusterPhi'].array()[presel]
    cscRechitClusterMuonVetoPt = T['cscRechitClusterMuonVetoPt'].array()[presel]
    cscRechitClusterMuonVetoGlobal = T['cscRechitClusterMuonVetoGlobal'].array()[presel]
    cscRechitClusterJetVetoPt = T['cscRechitClusterJetVetoPt'].array()[presel]
    cscRechitClusterDNN = T['cscRechitClusterDNN_bkgMC_plusBeamHalo'].array()[presel]

    muon_veto = np.logical_not((cscRechitClusterMuonVetoPt > 30) & cscRechitClusterMuonVetoGlobal)
    jet_veto = cscRechitClusterJetVetoPt < 30

    cond = cond & (np.abs(deltaPhi(np.array(cscRechitClusterPhi[:,0]),np.array(cscRechitClusterPhi[:,1]))) >1.8)
    print("dphi < 1.8",'\t', np.sum(weight[cond]))

    cond = cond & (np.sum(muon_veto, axis=1)==2)
    print("muon veto", '\t',np.sum(weight[cond]))

    cond = cond & (np.sum(jet_veto, axis=1)==2)
    print("jet veto", '\t',np.sum(weight[cond]))

    cond = cond & (np.sum(cscRechitClusterDNN>0.96, axis=1)==2)
    print("DNN",'\t', np.sum(weight[cond]))


sample 	 7791.838
acceptance 	 5622.125
L1 (at least one cluster) 	 4226.267
0 DT 	 4226.267
1 CSC-CSC 	 4091.208
CSC0IT 	 4079.5273
CSC1IT 	 3966.3691
CSC0 ME1 	 2874.212
CSC1 ME1 	 1571.8005
low MET 	 1518.5067
dphi < 1.8 	 1454.2622
muon veto 	 1449.1519
jet veto 	 1419.2197
DNN 	 1100.1875
sample 	 2550979.0
L1 (at least one cluster) 	 1195091.0
0 DT 	 1195091.0
1 CSC-CSC 	 1092400.0
CSC0IT 	 1089077.0
CSC1IT 	 73464.0
CSC0 ME1 	 64370.0
CSC1 ME1 	 6705.0
low MET 	 6704.0
dphi < 1.8 	 1262.0
muon veto 	 1250.0
jet veto 	 1103.0
DNN 	 106.0
CPU times: user 7.54 s, sys: 617 ms, total: 8.15 s
Wall time: 8.15 s


## CSC-DT

In [143]:
%%time

for k,T in files.items():
    if not k == 'mc':continue
    # weight = T['weight'].array()* 52*27000/NEVENTS
    if 'data' in k: weight = T['weight'].array()
    else: weight = T['weight'].array()* 1.1328524540090597e-06 * 27.82 * 1000 # from Pedro 2023
    #48/2.31671e+07= 2.0719037e-06?

        
    presel =  (T["nCscRechitClusters"].array() >= 0)
    # cond = T["HLTDecision"].array()[:,566] & (T["nCscRechitClusters"].array() > 1)
    ########### SELECTION: JETS ############

    print("sample", '\t', np.sum(weight[presel]))

    # csc = (T['gLLP_eta'].array()<3) & (T['gLLP_decay_vertex_r'].array()<800)\
    # & (T['gLLP_decay_vertex_z'].array()<1200) & (T['gLLP_decay_vertex_z'].array()>400)
    sel_csc = T['cscRechitCluster_match_gLLP'].array() & (np.abs(T['cscRechitCluster_match_gLLP_eta'].array())<3) & (T['cscRechitCluster_match_gLLP_decay_r'].array()<800)\
    & (np.abs(T['cscRechitCluster_match_gLLP_decay_z'].array())<1200) & (np.abs(T['cscRechitCluster_match_gLLP_decay_z'].array())>400)
    sel_dt = T['dtRechitCluster_match_gLLP'].array() &  (T['dtRechitCluster_match_gLLP_decay_r'].array()>200) &\
     (T['dtRechitCluster_match_gLLP_decay_r'].array()<800) &  (np.abs(T['dtRechitCluster_match_gLLP_decay_z'].array())<700)
    
    if not 'data' in k:
        presel = presel & (np.sum(dt, axis=1)>0) & (np.sum(csc, axis=1)>0)
        print("acceptance", '\t',np.sum(weight[presel]))

    cscRechitClusterR = (T['cscRechitClusterX'].array()**2+T['cscRechitClusterY'].array()**2)**0.5
    L1 = L1_trg(cscRechitClusterR, np.abs(T['cscRechitClusterZ'].array()), np.abs(T['cscRechitClusterSize'].array()))
    cscintime = (T['cscRechitClusterTimeWeighted'].array() < 12.5) & (T['cscRechitClusterTimeWeighted'].array()> -5) \
    & (T['cscRechitClusterTimeSpreadWeightedAll'].array() < 20)
    sel_csc = sel_csc & L1
    presel =presel & (np.sum(sel_csc, axis=1)>0)
    print("L1 (at least one cluster)", '\t',np.sum(weight[presel]))

    presel = presel & (T['met'].array()<150)
    print("low MET", '\t', np.sum(weight[presel]))
    
    sel_csc = sel_csc & cscintime
    presel = presel & (np.sum(cscintime, axis=1)>0)
    print("CSC intime", '\t', np.sum(weight[presel]))
    
    sel_dt = sel_dt & (T['dtRechitCluster_match_RPCBx_dPhi0p5'].array() <= 0) & (T['dtRechitCluster_match_RPChits_dPhi0p5'].array() >= 1)
    presel = presel & (np.sum(sel_dt, axis=1)>0)
    print("DT intime", '\t', np.sum(weight[presel]))
    
    sel_dt = sel_dt & (np.sum(T['dtRechitClusterNHitStation1'].array()==0, axis=1)>0)
    presel = presel & (np.sum(sel_dt, axis=1)>0)
    print("MB1 veto", '\t', np.sum(weight[presel]))


    cscRechitClusterJetVetoPt = T['cscRechitClusterJetVetoPt'].array()[sel_csc][presel][:,0]
    cscRechitClusterMuonVetoPt = T['cscRechitClusterMuonVetoPt'].array()[sel_csc][presel][:,0]
    cscRechitClusterMuonVetoGlobal = T['cscRechitClusterMuonVetoGlobal'].array()[sel_csc][presel][:,0]
    cscRechitClusterPhi = T['cscRechitClusterPhi'].array()[sel_csc][presel][:,0]
    cscRechitClusterDNN = T['cscRechitClusterDNN_bkgMC_plusBeamHalo'].array()[sel_csc][presel][:,0]
    
    dtRechitClusterJetVetoPt = T['dtRechitClusterJetVetoPt'].array()[sel_csc][presel][:,0]
    dtRechitClusterMuonVetoPt = T['dtRechitClusterMuonVetoPt'].array()[sel_csc][presel][:,0]
    dtRechitClusterMuonVetoLooseId = T['dtRechitClusterMuonVetoLooseId'].array()[sel_csc][presel][:,0]
    dtRechitClusterPhi = T['dtRechitClusterPhi'].array()[sel_csc][presel][:,0]
    weight = weight[presel]
    
    csc_muon_veto = np.logical_not((cscRechitClusterMuonVetoPt > 30) & cscRechitClusterMuonVetoGlobal)
    dt_muon_veto = np.logical_not((dtRechitClusterMuonVetoPt > 30) & dtRechitClusterMuonVetoLooseId)
    dt_jet_veto = (dtRechitClusterJetVetoPt < 30)
    csc_jet_veto = (cscRechitClusterJetVetoPt < 30)

    cond = csc_muon_veto & dt_muon_veto
    print("muon veto", '\t',np.sum(weight[cond]))

    cond = cond & dt_jet_veto & csc_jet_veto
    print("jet veto", '\t',np.sum(weight[cond]))


    
    cond = cond & (np.abs(deltaPhi(np.array(cscRechitClusterPhi),np.array(dtRechitClusterPhi))) >0.4)
    print("dphi < 1.8",'\t', np.sum(weight[cond]))

    cond = cond & (cscRechitClusterDNN>0.96)
    print("DNN",'\t', np.sum(weight[cond]))


sample 	 4275.9106
acceptance 	 4209.476
L1 (at least one cluster) 	 2287.981
low MET 	 2190.1543
CSC intime 	 2180.6633
DT intime 	 2096.7075
MB1 veto 	 1783.5156
muon veto 	 1768.9146
jet veto 	 1655.7566
dphi < 1.8 	 1637.5054
DNN 	 1469.5933
CPU times: user 155 ms, sys: 268 µs, total: 156 ms
Wall time: 154 ms


<Array [0, 0, 27.9, 16.2, 15.6, ... 0, 0, 0, 0] type='2443 * float32'>